In [1]:
import numpy as np
import pandas as pd
import os

from rdkit import Chem
from rdkit.Chem import MACCSkeys, rdFingerprintGenerator
from rdkit import DataStructs
from mordred import Calculator, descriptors

In [2]:
# endpoint = 'skin-sensitization'
endpoint = 'eye-irritation'

loc = r'D:\School\Semester3\Seminar - Reproducibility\seminar-toxicity\data'
endpoint_loc = os.path.join(loc, endpoint)

In [3]:
filename = 'data.csv'
df = pd.read_csv(os.path.join(endpoint_loc, filename))

In [4]:
df.head()

,CASRN,SMILES,Activity
0,51581-32-9,CN(C)C(=O)OC1C=CC=NC=1,1
1,35155-28-3,CN1C=C2CC3N(C)CC(CO)CC3(OC)C3C=CC=C1C=32,1
2,289-95-2,C1N=CC=CN=1,1
3,77-78-1,COS(=O)(=O)OC,1
4,80-73-9,CN1CCN(C)C1=O,1


#### MODI Index

For Binary classification

$$MODI = \frac{1}{2} \sum_{i=1}^2 \frac{N_i^{same}}{N_i^{total}}$$

In [5]:
def customTanimoto(a, b):
    return np.sum(a*b)/(np.sum(a**2) + np.sum(b**2) - np.sum(a*b))

def getMODIindex(df, fps, fp = 'rdkit'):
    '''
        df should contain the followinng columns
        SMILES - smiles string
        Activity - respective classification
    '''
    modified_df = df.copy()
    modified_df['fps'] = fps

    Modi = 0
    for activity in modified_df['Activity'].unique():
        temp_df = modified_df[modified_df['Activity'] == activity]
        Ni_same = 0
        Ni_total = 0
        for i in range(temp_df.shape[0]):
            sim = []
            if temp_df['fps'].iloc[i] is None:
                continue
            for j in range(modified_df.shape[0]):
                if modified_df['fps'].iloc[j] is None:
                    continue
                if temp_df['SMILES'].iloc[i] != modified_df['SMILES'].iloc[j]:
                    if fp == 'rdkit':
                        sim.append((DataStructs.TanimotoSimilarity(temp_df['fps'].iloc[i], modified_df['fps'].iloc[j]),
                                    temp_df['Activity'].iloc[i] == modified_df['Activity'].iloc[j]))
                    else:
                        sim.append((customTanimoto(temp_df['fps'].iloc[i], modified_df['fps'].iloc[j]), 
                                    temp_df['Activity'].iloc[i] == modified_df['Activity'].iloc[j]))
            
            max_element = max(sim, key = lambda x: x[0])

            if max_element[1]:
                Ni_same += 1
            Ni_total += 1

        Modi += Ni_same/Ni_total

    Modi = Modi/len(modified_df['Activity'].unique())

    return Modi

In [6]:
'''
    Using MACCS Fingerprints
'''
fps = []
for smiles in df['SMILES']:
    mol = Chem.MolFromSmiles(smiles)
    if mol is None:
        fps.append(None)
    else:
        fps.append(MACCSkeys.GenMACCSKeys(mol))

assert len(fps) == df.shape[0]

[22:14:25] Can't kekulize mol.  Unkekulized atoms: 0 1 2 3 4
[22:14:25] Can't kekulize mol.  Unkekulized atoms: 5 6 7 9 10 11 12 13 14 15 16 17 18
[22:14:25] Can't kekulize mol.  Unkekulized atoms: 10 11 12 13 15 16 17 19 20 21 25 27 28
[22:14:26] Can't kekulize mol.  Unkekulized atoms: 1 2 3 4 5 6 7 9 10 11 13 14 15
[22:14:26] Can't kekulize mol.  Unkekulized atoms: 10 11 12 13 14 15 16 18 19 20 21 22 23
[22:14:26] Can't kekulize mol.  Unkekulized atoms: 5 6 7 9 10 11 12 13 14 15 16 17 18


In [7]:
Modi = getMODIindex(df, fps)
print('Modi for MACCS Keys :', Modi)

Modi for MACCS Keys : 0.6918367346938776


In [8]:
'''
    Using Morgan Fingerprints r = 3 and nbits = 2048
'''
fpg = rdFingerprintGenerator.GetMorganGenerator(radius=3, fpSize=2048)
fps = []
for smiles in df['SMILES']:
    mol = Chem.MolFromSmiles(smiles)
    if mol is None:
        fps.append(None)
    else:
        fps.append(fpg.GetFingerprint(mol))

assert len(fps) == df.shape[0]

[22:30:20] Can't kekulize mol.  Unkekulized atoms: 0 1 2 3 4
[22:30:20] Can't kekulize mol.  Unkekulized atoms: 5 6 7 9 10 11 12 13 14 15 16 17 18
[22:30:20] Can't kekulize mol.  Unkekulized atoms: 10 11 12 13 15 16 17 19 20 21 25 27 28
[22:30:20] Can't kekulize mol.  Unkekulized atoms: 1 2 3 4 5 6 7 9 10 11 13 14 15
[22:30:20] Can't kekulize mol.  Unkekulized atoms: 10 11 12 13 14 15 16 18 19 20 21 22 23
[22:30:20] Can't kekulize mol.  Unkekulized atoms: 5 6 7 9 10 11 12 13 14 15 16 17 18


In [9]:
Modi = getMODIindex(df, fps)
print('Modi for Morgan fingerprints :', Modi)

Modi for Morgan fingerprints : 0.700597127739985


In [ ]:
'''
    Using Mordred Fingerprints
'''
calc = Calculator(descriptors, ignore_3D=True)
fps = []
for smiles in df['SMILES']:
    mol = Chem.MolFromSmiles(smiles)
    if mol is None:
        fps.append(None)
    else:
        fps.append(np.array(calc(mol)))

assert len(fps) == df.shape[0]

In [ ]:
customTanimoto(fps[0], fps[1])

In [ ]:
Modi = getMODIindex(df, fps, fp='custom')
print('Modi for Mordred fingerprints :', Modi)

##### Skin Sensitization

Modi index for Maccs keys = 0.6629 \
Modi index for Morgan keys = 0.6571 \
Modi index for Mordred keys = -

##### Eye irritation

Modi index for Maccs keys = 0.6918 \
Modi index for Morgan keys = 0.7006 \
Modi index for Mordred keys = 